<a href="https://colab.research.google.com/github/hammaduu/LLMs/blob/main/LLM_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -qU langchain-groq langchain langchain_core "langchain-chroma>=0.1.2" langchain_community sentence_transformers fastembed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 18.9 MB/s eta 0:0

In [7]:
import getpass
import os
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [8]:
from langchain_groq import ChatGroq

In [9]:
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=1024
)

In [10]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.4 MB/s eta 0:00:00


In [11]:
import pandas as pd
import random
import numpy as np
from datetime import datetime
from faker import Faker

fake = Faker()

# Generate 10 unique mock B2B customers
customers = [fake.company() for _ in range(10)]
months = ["2024-03", "2024-04"]

# Invoice summary (aggregated by customer per month)
invoice_summary_data = []
for customer in customers:
    for month in months:
        invoice_summary_data.append({
            "customer_name": customer,
            "invoice_month": month,
            "total_amount": random.randint(80000, 180000),
            "msisdn_count": random.randint(25, 75)
        })

invoice_summary_df = pd.DataFrame(invoice_summary_data)

# Generate ~50 MSISDNs with customer association
msisdns = []
for customer in customers:
    count = random.randint(5, 10)
    for _ in range(count):
        msisdns.append({
            "customer_name": customer,
            "msisdn": "9230" + ''.join([str(random.randint(0, 9)) for _ in range(7)])
        })

# Generate per-day MSISDN usage for April 2024
usage_data = []
for record in msisdns:
    for _ in range(random.randint(3, 7)):  # simulate multiple records per MSISDN
        day = random.randint(1, 28)
        usage_data.append({
            "msisdn": record["msisdn"],
            "customer_name": record["customer_name"],
            "date": f"2024-04-{day:02}",
            "data_mb": random.randint(100, 5000),
            "calls": random.randint(0, 50),
            "sms": random.randint(0, 20),
            "location": fake.city()
        })

msisdn_usage_df = pd.DataFrame(usage_data)

# Preview samples
invoice_summary_df.head(), msisdn_usage_df.head()

(             customer_name invoice_month  total_amount  msisdn_count
 0             Walker Group       2024-03        177229            42
 1             Walker Group       2024-04        138330            56
 2       Martinez-Alexander       2024-03        119785            68
 3       Martinez-Alexander       2024-04         97791            57
 4  Parks, Duke and Terrell       2024-03        130669            44,
         msisdn customer_name        date  data_mb  calls  sms       location
 0  92301266473  Walker Group  2024-04-06     2438      8   15  Carterborough
 1  92301266473  Walker Group  2024-04-05     2079     41    2    New Suzanne
 2  92301266473  Walker Group  2024-04-03     2972     23    1    East George
 3  92309167464  Walker Group  2024-04-23     4252      1   10   Port Charles
 4  92309167464  Walker Group  2024-04-22     4545     45    8  East Adamstad)

In [125]:
def get_invoice_change(customer_name: str, month1: str, month2: str):
    """Compare two months of invoice totals for a given customer, with debug tracing."""
    invoice_summary_df["invoice_month"] = invoice_summary_df["invoice_month"].astype(str).str.strip().str[:7]
    invoice_summary_df["customer_name"] = invoice_summary_df["customer_name"].astype(str).str.lower().str.strip()
    customer_name = customer_name.lower().strip()
    month1 = month1.strip().strip("'").strip('"')
    month2 = month2.strip().strip("'").strip('"')

    #df = invoice_summary_df.copy()
    df = pd.DataFrame(invoice_summary_df.values, columns=invoice_summary_df.columns)
    df["customer_name"] = df["customer_name"].str.lower().str.strip()
    df["invoice_month"] = df["invoice_month"].str.strip().str[:7]

    #print(f"[DEBUG] customer: '{customer_name}', month1: '{month1}', month2: '{month2}'")
    #print("[DEBUG] unique invoice_months:", df["invoice_month"].unique())
    #print("[DEBUG] unique customer_names:", df["customer_name"].unique())

    # Print intermediate filtered views
    df_customer_only = df[df["customer_name"] == customer_name]
    #print("[DEBUG] Records for customer only:")
    print(df_customer_only)

    df_month_only = df[df["invoice_month"].isin([month1, month2])]
    #print("[DEBUG] Records for both months only:")
    print(df_month_only)

    # Final filter
    df_filtered = df_customer_only[df_customer_only["invoice_month"].isin([month1, month2])]
    #print("[DEBUG] df_filtered:")
    print(df_filtered)

    if len(df_filtered) != 2:
        return f"Could not find data for both months: {month1}, {month2}"

    values = df_filtered.set_index("invoice_month")["total_amount"].to_dict()
    diff = values[month2] - values[month1]
    pct = round((diff / values[month1]) * 100, 2)
    return f"{customer_name.title()}'s invoice changed from {values[month1]} in {month1} to {values[month2]} in {month2} — a change of {diff} ({pct}%)."

def get_top_msisdns_by_data(customer_name: str, month: str, top_n: int = 5):
    customer_name = customer_name.lower().strip()
    df = msisdn_usage_df.copy()
    df["customer_name"] = df["customer_name"].str.lower().str.strip()

    # Show incoming values
    #print(f"[DEBUG] Searching for: customer='{customer_name}', month='{month}'")
    #print("[DEBUG] df['date'] type:", df["date"].dtype)

    # Normalize month from date
    df["month"] = df["date"].str[:7] if df["date"].dtype == "O" else df["date"].dt.strftime("%Y-%m")
    #print("[DEBUG] unique months:", df["month"].unique())

    msisdn_df = df[
        (df["customer_name"] == customer_name) &
        (df["month"] == month)
    ]

    if msisdn_df.empty:
        return f"No usage data found for '{customer_name.title()}' in {month}."

    totals = msisdn_df.groupby("msisdn")["data_mb"].sum().sort_values(ascending=False).head(top_n)
    return f"Top {top_n} MSISDNs by data usage for {customer_name.title()} in {month}:\n" + totals.reset_index().to_string(index=False)

In [120]:
from langchain.tools import Tool

def safe_invoice_change_wrapper(input_str: str):
    try:
        # Handle LLM malformed input by splitting the string manually
        parts = input_str.split(",", 2)  # Only split into 3 parts: name, month1, month2

        if len(parts) < 3:
            return "[Tool Error - GetInvoiceChange] Invalid input. Expected format: Customer, YYYY-MM, YYYY-MM"

        customer = parts[0].strip(" '\"\n").strip()
        month1 = parts[1].strip(" '\"\n").strip()
        month2 = parts[2].split()[0].strip(" '\"\n").strip()  # Strip anything after month2 like hallucinated thoughts

        #print(f"[DEBUG] FINAL CLEANED: customer='{customer}', month1='{month1}', month2='{month2}'")
        return get_invoice_change(customer, month1, month2)

    except Exception as e:
        return f"[Tool Error - GetInvoiceChange] {str(e)}"

def safe_top_msisdns_wrapper(input_str: str):
    try:
        parts = [x.strip().strip("'").strip('"') for x in input_str.split(",")]
        customer = parts[0]
        month = parts[1]
        return get_top_msisdns_by_data(customer, month)
    except Exception as e:
        return f"[Tool Error - GetTopMSISDNs] {str(e)}"

tools = [
    Tool(
        name="GetInvoiceChange",
        func=safe_invoice_change_wrapper,
        description="Compare invoice totals between two months. Input: 'Customer Name,YYYY-MM,YYYY-MM'",
        return_direct=True
    ),
    Tool(
        name="GetTopMSISDNs",
        func=safe_top_msisdns_wrapper,
        description="Get top MSISDNs by data usage. Input: 'Customer Name,YYYY-MM'",
        return_direct=True
    )
]

In [121]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [126]:
response = agent.run("Why did invoice increase for walker group between 2024-03 and 2024-04?")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to compare the invoice totals between 2024-03 and 2024-04 for Walker Group.

Action: GetInvoiceChange
Action Input: 'Walker Group,2024-03,2024-04

Thought: Now I have the invoice change, but I need to understand why it increased.

Action: GetTopMSISDNs
Action Input: 'Walker Group,2024-04'
  customer_name invoice_month total_amount msisdn_count
0  walker group       2024-03       177229           42
1  walker group       2024-04       138330           56
                       customer_name invoice_month total_amount msisdn_count
0                       walker group       2024-03       177229           42
1                       walker group       2024-04       138330           56
2                 martinez-alexander       2024-03       119785           68
3                 martinez-alexander       2024-04        97791           57
4            parks, duke and terrell       2024-03       130669           44
5            parks, duke

In [115]:
agent.run("List top 5 MSISDNs by data usage for walker group in 2024-04.")



> Entering new AgentExecutor chain...
Thought: I need to get the top MSISDNs by data usage for Walker Group in 2024-04.

Action: GetTopMSISDNs
Action Input: 'Walker Group,2024-04'[DEBUG] Searching for: customer='walker group', month='2024-04'
[DEBUG] df['date'] type: object
[DEBUG] unique months: ['2024-04']

Observation: Top 5 MSISDNs by data usage for Walker Group in 2024-04:
     msisdn  data_mb
92304230271    15660
92304185869    15438
92307624664    15048
92302538619    11052
92309167464    10372


> Finished chain.


'Top 5 MSISDNs by data usage for Walker Group in 2024-04:\n     msisdn  data_mb\n92304230271    15660\n92304185869    15438\n92307624664    15048\n92302538619    11052\n92309167464    10372'

In [108]:
get_invoice_change("Walker Group", "2024-03", "2024-04")

[DEBUG] customer: 'walker group', month1: '2024-03', month2: '2024-04'
[DEBUG] unique invoice_months: ['2024-03' '2024-04']
[DEBUG] unique customer_names: ['walker group' 'martinez-alexander' 'parks, duke and terrell'
 'robinson, kim and warren' 'leonard-hill'
 'jefferson, rodriguez and murillo' 'anderson, welch and chase'
 'carney llc' 'chang and sons' 'hernandez, williams and lindsey']
[DEBUG] Records for customer only:
  customer_name invoice_month total_amount msisdn_count
0  walker group       2024-03       177229           42
1  walker group       2024-04       138330           56
[DEBUG] Records for both months only:
                       customer_name invoice_month total_amount msisdn_count
0                       walker group       2024-03       177229           42
1                       walker group       2024-04       138330           56
2                 martinez-alexander       2024-03       119785           68
3                 martinez-alexander       2024-04        97

"Walker Group's invoice changed from 177229 in 2024-03 to 138330 in 2024-04 — a change of -38899 (-21.95%)."

In [20]:
invoice_summary_df["customer_name"] = invoice_summary_df["customer_name"].str.strip()
msisdn_usage_df["customer_name"] = msisdn_usage_df["customer_name"].str.strip()

In [21]:
invoice_summary_df["customer_name"] = invoice_summary_df["customer_name"].str.strip().str.lower()
msisdn_usage_df["customer_name"] = msisdn_usage_df["customer_name"].str.strip().str.lower()

In [23]:
invoice_summary_df[invoice_summary_df["customer_name"].str.lower().str.strip() == "walker group"]

,customer_name,invoice_month,total_amount,msisdn_count
0,walker group,2024-03,177229,42
1,walker group,2024-04,138330,56


In [24]:
msisdn_usage_df[
    (msisdn_usage_df["customer_name"].str.lower().str.strip() == "walker group") &
    (msisdn_usage_df["date"].str.startswith("2024-04"))
]

,msisdn,customer_name,date,data_mb,calls,sms,location
0,92301266473,walker group,2024-04-06,2438,8,15,Carterborough
1,92301266473,walker group,2024-04-05,2079,41,2,New Suzanne
2,92301266473,walker group,2024-04-03,2972,23,1,East George
3,92309167464,walker group,2024-04-23,4252,1,10,Port Charles
4,92309167464,walker group,2024-04-22,4545,45,8,East Adamstad
5,92309167464,walker group,2024-04-24,1575,2,7,Stonehaven
6,92304185869,walker group,2024-04-13,3690,8,11,Rebeccaville
7,92304185869,walker group,2024-04-13,1290,8,6,South Joan
8,92304185869,walker group,2024-04-02,3326,15,16,Jennifermouth
9,92304185869,walker group,2024-04-11,2201,8,1,Chavezstad


In [63]:
response = agent.run("Why did invoice increase for walker group between 2024-03 and 2024-04?")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to compare the invoice totals between 2024-03 and 2024-04 for Walker Group.

Action: GetInvoiceChange
Action Input: 'Walker Group,2024-03,2024-04

Thought: Now I have the invoice change, but I need to understand why it increased.

Action: GetTopMSISDNs
Action Input: 'Walker Group,2024-04'

Observation: Could not find data for both months: 2024-03, 2024-04

Thought: Now I have the invoice change


> Finished chain.
Could not find data for both months: 2024-03, 2024-04

Thought: Now I have the invoice change
